# Lab: Introduction to LangGraph - Building Your First Agentic Graph

This lab introduces **LangGraph**, a library from the creators of LangChain for building robust, stateful agentic applications. LangGraph allows you to define agent workflows as graphs, where each node is a step in the process and edges define the flow of control. This is particularly useful for creating complex agents that can loop, branch, and call tools in sophisticated ways.

We will learn the five core steps of building a LangGraph application:
1.  **Define the State**: Create an object that will hold the application's state as it moves through the graph.
2.  **Create the Graph Builder**: Initialize the graph with the state object.
3.  **Add Nodes**: Define the functions that will perform the work at each step.
4.  **Add Edges**: Connect the nodes to define the workflow.
5.  **Compile the Graph**: Create the final, runnable application.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# === Imports ===
from typing import Annotated
from dotenv import load_dotenv
import random

# Core LangGraph components
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

# For building the chatbot UI
from IPython.display import Image, display
import gradio as gr

# For defining state and interacting with LLMs
from langchain_openai import ChatOpenAI
from pydantic import BaseModel

In [ ]:
load_dotenv(override=True)

### Part 1: Building a Graph with Simple Python (No LLM)

To understand the core mechanics of LangGraph, we'll first build a graph that doesn't involve any AI. This will help isolate the concepts of state, nodes, and edges.

#### Step 1: Define the State Object

The `State` is a central concept in LangGraph. It's a Python object that holds all the data that needs to be passed between the nodes in our graph. For a chatbot, this is typically the list of messages in the conversation.

We use `Annotated` and `add_messages` to tell LangGraph how to update the `messages` field. `add_messages` is a built-in function (a "reducer") that correctly appends new messages to the existing list, maintaining the conversation history.

In [ ]:
# The State class defines the structure of our application's state.
# Here, it contains a single field, `messages`.
class SimpleState(BaseModel):
    messages: Annotated[list, add_messages]

#### Step 2: Create the Graph Builder

We initialize a `StateGraph` and pass it our state definition. This builder object is what we'll use to construct our workflow.

In [ ]:
graph_builder = StateGraph(SimpleState)

#### Step 3: Add a Node

A node is a function that performs an action. It receives the current state of the graph as input and should return a new state object with any updates. Here, our node will generate a random, silly phrase.

In [ ]:
# Some fun constants for our simple node
nouns = ["Cabbages", "Unicorns", "Toasters", "Penguins", "Bananas"]
adjectives = ["outrageous", "smelly", "pedantic", "existential", "moody"]

def silly_node(state: SimpleState) -> SimpleState:
    """A simple function that acts as a node in our graph."""
    # This node ignores the input state and just generates a new message.
    reply = f"{random.choice(nouns)} are {random.choice(adjectives)}."
    # It returns a new State object with the assistant's reply.
    return SimpleState(messages=[{"role": "assistant", "content": reply}])

# We add the node to our graph builder with a unique name.
graph_builder.add_node("silly_node", silly_node)

#### Step 4: Add Edges

Edges define the path of execution. We need to tell the graph where to start and where to go after each node. `START` and `END` are special keywords.
- The first edge connects the `START` of the graph to our `silly_node`.
- The second edge connects `silly_node` to the `END`, meaning the graph will terminate after this node runs.

In [ ]:
graph_builder.add_edge(START, "silly_node")
graph_builder.add_edge("silly_node", END)

#### Step 5: Compile the Graph

This finalizes the graph and makes it a runnable object. We can also visualize the structure we've just built.

In [ ]:
simple_graph = graph_builder.compile()

# Display a visual representation of the graph.
display(Image(simple_graph.get_graph().draw_mermaid_png()))

In [ ]:
# Let's test our simple graph.
initial_state = SimpleState(messages=[{"role": "user", "content": "Hello!"}])
result = simple_graph.invoke(initial_state)

print(result['messages'][-1]['content'])

### Part 2: Building a Chatbot with an LLM

Now that we understand the 5-step process, let's replace our simple Python node with a node that calls an LLM. The structure of the graph will be identical.

In [ ]:
# Step 1 & 2: Define State and initialize the Graph Builder
class ChatState(BaseModel):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(ChatState)

In [ ]:
# Step 3: Create a Node that calls an LLM
llm = ChatOpenAI(model="gpt-4o-mini")

def chatbot_node(state: ChatState) -> ChatState:
    """This node invokes the LLM with the current conversation history."""
    # The `state.messages` contains the full history, which is passed to the LLM.
    response = llm.invoke(state.messages)
    # We return a new state object containing only the LLM's latest response.
    # `add_messages` will handle appending it to the history.
    return ChatState(messages=[response])

graph_builder.add_node("chatbot", chatbot_node)

In [ ]:
# Step 4: Create Edges
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

In [ ]:
# Step 5: Compile the Graph
chatbot_graph = graph_builder.compile()
display(Image(chatbot_graph.get_graph().draw_mermaid_png()))

### Showtime! Launching the Chatbot UI

Now we can connect our compiled graph to a Gradio user interface to create a fully functional chatbot.

In [ ]:
def chat_interface_function(user_input: str, history: list):
    """The function that Gradio will call on each user interaction."""
    # The initial state for the graph is just the user's new message.
    initial_state = ChatState(messages=[{"role": "user", "content": user_input}])
    
    # We invoke the graph, which runs the chatbot node.
    result = chatbot_graph.invoke(initial_state)
    
    # The final message from the assistant is returned to the UI.
    return result['messages'][-1].content

gr.ChatInterface(
    chat_interface_function, 
    title="My First LangGraph Chatbot",
    description="A simple chatbot built using the 5 core steps of LangGraph."
).launch()